In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit


## Arithmetical features

- To practice creating new features, you will be working with a subsample from the Kaggle competition called "House Prices: Advanced Regression Techniques". The goal of this competition is to predict the price of the house based on its properties. It's a regression problem with Root Mean Squared Error as an evaluation metric.

- Your goal is to create new features and determine whether they improve your validation score. To get the validation score from 5-fold cross-validation, you're given the get_kfold_rmse() function. Use it with the train DataFrame, available in your workspace, as an argument.

### Instructions 1/3
-    Create a new feature representing the total area (basement, 1st and 2nd floors) of the house. The columns "TotalBsmtSF", "FirstFlrSF" and "SecondFlrSF" give the areas of the basement, 1st and 2nd floors, 

In [14]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

kf = KFold()

def get_kfold_rmse(train):
    mse_scores = []

    for train_index, test_index in kf.split(train):
        train = train.fillna(0)
        feats = [x for x in train.columns if x not in ['Id', 'SalePrice', 'RoofStyle', 'CentralAir']]
        
        fold_train, fold_test = train.loc[train_index], train.loc[test_index]

        # Fit the data and make predictions
        # Create a Random Forest object
        rf = RandomForestRegressor(n_estimators=10, min_samples_split=10, random_state=123)

        # Train a model
        rf.fit(X=fold_train[feats], y=fold_train['SalePrice'])

        # Get predictions for the test set
        pred = rf.predict(fold_test[feats])
    
        fold_score = mean_squared_error(fold_test['SalePrice'], pred)
        mse_scores.append(np.sqrt(fold_score))
        
    return round(np.mean(mse_scores) + np.std(mse_scores), 2)

train = pd.read_csv("../../data/house_prices_train.csv")
test = pd.read_csv("../../data/house_prices_test.csv")

train["FirstFlrSF"] = train["1stFlrSF"]
train["SecondFlrSF"] = train["2ndFlrSF"]

test["FirstFlrSF"] = test["1stFlrSF"]
test["SecondFlrSF"] = test["2ndFlrSF"]

In [15]:
# Look at the initial RMSE
print('RMSE before feature engineering:', get_kfold_rmse(train))

# Find the total area of the house
train['TotalArea'] = train["TotalBsmtSF"] + train["FirstFlrSF"] + train["SecondFlrSF"]

# Look at the updated RMSE
print('RMSE with total area:', get_kfold_rmse(train))

RMSE before feature engineering: 34458.13
RMSE with total area: 34180.15


### Instructions 2/3
-    Create a new feature representing the area of the garden. It is a difference between the total area of the property ("LotArea") and the first floor area ("FirstFlrSF").

In [16]:
# Look at the initial RMSE
print('RMSE before feature engineering:', get_kfold_rmse(train))

# Find the total area of the house
train['TotalArea'] = train['TotalBsmtSF'] + train['FirstFlrSF'] + train['SecondFlrSF']
print('RMSE with total area:', get_kfold_rmse(train))

# Find the area of the garden
train['GardenArea'] = train['LotArea'] - train['FirstFlrSF']
print('RMSE with garden area:', get_kfold_rmse(train))

RMSE before feature engineering: 34180.15
RMSE with total area: 34180.15
RMSE with garden area: 35301.11


### Instructions 3/3
-    Create a new feature representing the total number of bathrooms in the house. It is a sum of full bathrooms ("FullBath") and half bathrooms ("HalfBath").

In [17]:
# Look at the initial RMSE
print('RMSE before feature engineering:', get_kfold_rmse(train))

# Find the total area of the house
train['TotalArea'] = train['TotalBsmtSF'] + train['FirstFlrSF'] + train['SecondFlrSF']
print('RMSE with total area:', get_kfold_rmse(train))

# Find the area of the garden
train['GardenArea'] = train['LotArea'] - train['FirstFlrSF']
print('RMSE with garden area:', get_kfold_rmse(train))

# Find total number of bathrooms
train['TotalBath'] = train['FullBath'] + train['HalfBath']
print('RMSE with number of bathrooms:', get_kfold_rmse(train))

RMSE before feature engineering: 35301.11
RMSE with total area: 35301.11
RMSE with garden area: 35301.11
RMSE with number of bathrooms: 34920.65


## Date features

- You've built some basic features using numerical variables. Now, it's time to create features based on date and time. You will practice on a subsample from the Taxi Fare Prediction Kaggle competition data. The data represents information about the taxi rides and the goal is to predict the price for each ride.

- Your objective is to generate date features from the pickup datetime. Recall that it's better to create new features for train and test data simultaneously. After the features are created, split the data back into the train and test DataFrames. Here it's done using pandas' isin() method.

- The train and test DataFrames are already available in your workspace.

### Instructions
-    Concatenate the train and test DataFrames into a single DataFrame taxi.
-    Convert the "pickup_datetime" column to a datetime object.
-    Create the day of week (using .dayofweek attribute) and hour (using .hour attribute) features from the "pickup_datetime" column.

In [18]:
train = pd.read_csv("../../data/taxi_train_chapter_4.csv")
test = pd.read_csv("../../data/taxi_test_chapter_4.csv")

In [19]:
# Concatenate train and test together
taxi = pd.concat([train, test])

# Convert pickup date to datetime object
taxi['pickup_datetime'] = pd.to_datetime(taxi['pickup_datetime'])

# Create a day of week feature
taxi['dayofweek'] = taxi['pickup_datetime'].dt.dayofweek

# Create an hour feature
taxi['hour'] = taxi['pickup_datetime'].dt.hour

# Split back into train and test
new_train = taxi[taxi['id'].isin(train['id'])]
new_test = taxi[taxi['id'].isin(test['id'])]

## Label encoding

- Let's work on categorical variables encoding. You will again work with a subsample from the House Prices Kaggle competition.

- Your objective is to encode categorical features "RoofStyle" and "CentralAir" using label encoding. The train and test DataFrames are already available in your workspace.

### Instructions
-    Concatenate train and test DataFrames into a single DataFrame houses.
-    Create a LabelEncoder object without arguments and assign it to le.
-    Create new label-encoded features for "RoofStyle" and "CentralAir" using the same le object.

In [21]:
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv("../../data/house_prices_train.csv")
test = pd.read_csv("../../data/house_prices_test.csv")

In [22]:
# Concatenate train and test together
houses = pd.concat([train, test])

# Label encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Create new features
houses['RoofStyle_enc'] = le.fit_transform(houses["RoofStyle"])
houses['CentralAir_enc'] = le.fit_transform(houses["CentralAir"])

# Look at new features
print(houses[['RoofStyle', 'RoofStyle_enc', 'CentralAir', 'CentralAir_enc']].head())

  RoofStyle  RoofStyle_enc CentralAir  CentralAir_enc
0     Gable              1          Y               1
1     Gable              1          Y               1
2     Gable              1          Y               1
3     Gable              1          Y               1
4     Gable              1          Y               1


## One-Hot encoding

- The problem with label encoding is that it implicitly assumes that there is a ranking dependency between the categories. So, let's change the encoding method for the features "RoofStyle" and "CentralAir" to one-hot encoding. Again, the train and test DataFrames from House Prices Kaggle competition are already available in your workspace.

- Recall that if you're dealing with binary features (categorical features with only two categories) it is suggested to apply label encoder only.

- Your goal is to determine which of the mentioned features is not binary, and to apply one-hot encoding only to this one.

### Instructions 1/4
-    Determine the distribution of "RoofStyle" and "CentralAir" features using pandas' value_counts() method.

In [23]:
# Concatenate train and test together
houses = pd.concat([train, test])

# Look at feature distributions
print(houses['RoofStyle'].value_counts(), '\n')
print(houses['CentralAir'].value_counts())

RoofStyle
Gable      2310
Hip         551
Gambrel      22
Flat         20
Mansard      11
Shed          5
Name: count, dtype: int64 

CentralAir
Y    2723
N     196
Name: count, dtype: int64


### Instructions 3/4
-    As long as "CentralAir" is a binary feature, encode it with a label encoder (0 - for one class and 1 - for another class).

In [24]:
# Concatenate train and test together
houses = pd.concat([train, test])

# Label encode binary 'CentralAir' feature
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
houses['CentralAir_enc'] = le.fit_transform(houses["CentralAir"])

### Instructions 4/4
-    For the categorical feature "RoofStyle" let's use the one-hot encoder. Firstly, create one-hot encoded features using the get_dummies() method. Then they are concatenated to the initial houses DataFrame.

In [25]:
# Concatenate train and test together
houses = pd.concat([train, test])

# Label encode binary 'CentralAir' feature
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
houses['CentralAir_enc'] = le.fit_transform(houses['CentralAir'])

# Create One-Hot encoded features
ohe = pd.get_dummies(houses['RoofStyle'], prefix='RoofStyle')

# Concatenate OHE features to houses
houses = pd.concat([houses, ohe], axis=1)

# Look at OHE features
print(houses[[col for col in houses.columns if 'RoofStyle' in col]].head(3))

  RoofStyle  RoofStyle_Flat  RoofStyle_Gable  RoofStyle_Gambrel  \
0     Gable           False             True              False   
1     Gable           False             True              False   
2     Gable           False             True              False   

   RoofStyle_Hip  RoofStyle_Mansard  RoofStyle_Shed  
0          False              False           False  
1          False              False           False  
2          False              False           False  


## Mean target encoding

- First of all, you will create a function that implements mean target encoding. Remember that you need to develop the two following steps:

    1.    Calculate the mean on the train, apply to the test
    2.    Split train into K folds. Calculate the out-of-fold mean for each fold, apply to this particular fold
<br/>
- Each of these steps will be implemented in a separate function: test_mean_target_encoding() and train_mean_target_encoding(), respectively.

- The final function mean_target_encoding() takes as arguments: the train and test DataFrames, the name of the categorical column to be encoded, the name of the target column and a smoothing parameter alpha. It returns two values: a new feature for train and test DataFrames, respectively.

### Instructions 1/3
-    You need to add smoothing to avoid overfitting. So, add $\alpha$ parameter to the denominator in train_statistics calculations.
-    You need to treat new categories in the test data. So, pass a global mean as an argument to the fillna() method.

In [26]:
def test_mean_target_encoding(train, test, target, categorical, alpha=5):
    # Calculate global mean on the train data
    global_mean = train[target].mean()
    
    # Group by the categorical feature and calculate its properties
    train_groups = train.groupby(categorical)
    category_sum = train_groups[target].sum()
    category_size = train_groups.size()
    
    # Calculate smoothed mean target statistics
    train_statistics = (category_sum + global_mean * alpha) / (category_size + alpha)
    
    # Apply statistics to the test data and fill new categories
    test_feature = test[categorical].map(train_statistics).fillna(global_mean)
    return test_feature.values

### Instructions 2/3
-    To calculate the train mean encoded feature you need to use out-of-fold statistics, splitting train into several folds. Specify the train and test indices for each validation split to access it.

In [27]:
def train_mean_target_encoding(train, target, categorical, alpha=5):
    # Create 5-fold cross-validation
    kf = KFold(n_splits=5, random_state=123, shuffle=True)
    train_feature = pd.Series(index=train.index)
    
    # For each folds split
    for train_index, test_index in kf.split(train):
        cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
      
        # Calculate out-of-fold statistics and apply to cv_test
        cv_test_feature = test_mean_target_encoding(cv_train, cv_test, target, categorical, alpha)
        
        # Save new feature for this particular fold
        train_feature.iloc[test_index] = cv_test_feature       
    return train_feature.values

### Instructions 3/3
-    Finally, you just calculate train and test target mean encoded features and return them from the function. So, return train_feature and test_feature obtained.

In [28]:
def mean_target_encoding(train, test, target, categorical, alpha=5):
  
    # Get the train feature
    train_feature = train_mean_target_encoding(train, target, categorical, alpha)
  
    # Get the test feature
    test_feature = test_mean_target_encoding(train, test, target, categorical, alpha)
    
    # Return new features to add to the model
    return train_feature, test_feature

## K-fold cross-validation

- You will work with a binary classification problem on a subsample from Kaggle playground competition. The objective of this competition is to predict whether a famous basketball player Kobe Bryant scored a basket or missed a particular shot.

- Train data is available in your workspace as bryant_shots DataFrame. It contains data on 10,000 shots with its properties and a target variable "shot\_made\_flag" -- whether shot was scored or not.

- One of the features in the data is "game_id" -- a particular game where the shot was made. There are 541 distinct games. So, you deal with a high-cardinality categorical feature. Let's encode it using a target mean!

- Suppose you're using 5-fold cross-validation and want to evaluate a mean target encoded feature on the local validation.

### Instructions
-    To achieve this, you need to repeat encoding procedure for the "game_id" categorical feature inside each folds split separately. Your goal is to specify all the missing parameters for the mean_target_encoding() function call inside each folds split.
-    Recall that the train and test parameters expect the train and test DataFrames.
-    While the target and categorical parameters expect names of the target variable and categorical feature to be encoded.

In [29]:
bryant_shots = pd.read_csv("../../data/bryant_shots_data.csv")

In [30]:
# Create 5-fold cross-validation
kf = KFold(n_splits=5, random_state=123, shuffle=True)

# For each folds split
for train_index, test_index in kf.split(bryant_shots):
    cv_train, cv_test = bryant_shots.iloc[train_index], bryant_shots.iloc[test_index]

    # Create mean target encoded feature
    cv_train['game_id_enc'], cv_test['game_id_enc'] = mean_target_encoding(train=cv_train,
                                                                           test=cv_test,
                                                                           target='shot_made_flag',
                                                                           categorical='game_id',
                                                                           alpha=5)
    # Look at the encoding
    print(cv_train[['game_id', 'shot_made_flag', 'game_id_enc']].sample(n=1))

       game_id  shot_made_flag  game_id_enc
4364  20200774             1.0      0.39249
        game_id  shot_made_flag  game_id_enc
17264  21000610             1.0     0.403392
        game_id  shot_made_flag  game_id_enc
17920  21001139             NaN     0.464476
       game_id  shot_made_flag  game_id_enc
2653  20100816             1.0     0.222784
        game_id  shot_made_flag  game_id_enc
12249  20700572             1.0     0.366376


/tmp/ipykernel_1054596/2144274239.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv_train['game_id_enc'], cv_test['game_id_enc'] = mean_target_encoding(train=cv_train,
/tmp/ipykernel_1054596/2144274239.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv_train['game_id_enc'], cv_test['game_id_enc'] = mean_target_encoding(train=cv_train,
/tmp/ipykernel_1054596/2144274239.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

## Beyond binary classification

- Of course, binary classification is just a single special case. Target encoding could be applied to any target variable type:
    -    For binary classification usually mean target encoding is used
    -    For regression mean could be changed to median, quartiles, etc.
    -    For multi-class classification with N classes we create N features with target mean for each category in one vs. all fashion
<br/>
- The mean_target_encoding() function you've created could be used for any target type specified above. Let's apply it for the regression problem on the example of House Prices Kaggle competition.

- Your goal is to encode a categorical feature "RoofStyle" using mean target encoding. The train and test DataFrames are already available in your workspace.

### Instructions
-    Specify all the missing parameters for the mean_target_encoding() function call. Target variable name is "SalePrice". Set $\alpha$ hyperparameter to 10.
-    Recall that the train and test parameters expect the train and test DataFrames.
-    While the target and categorical parameters expect names of the target variable and feature to be encoded.

In [31]:
train = pd.read_csv("../../data/house_prices_train.csv")
test = pd.read_csv("../../data/house_prices_test.csv")

In [32]:
# Create mean target encoded feature
train['RoofStyle_enc'], test['RoofStyle_enc'] = mean_target_encoding(train=train,
                                                                     test=test,
                                                                     target='SalePrice',
                                                                     categorical='RoofStyle',
                                                                     alpha=10)

# Look at the encoding
print(test[['RoofStyle', 'RoofStyle_enc']].drop_duplicates())

     RoofStyle  RoofStyle_enc
0        Gable  171565.947836
1          Hip  217594.645131
98     Gambrel  164152.950424
133       Flat  188703.563431
362    Mansard  180775.938759
1053      Shed  188267.663242


## Find missing data

- Let's impute missing data on a real Kaggle dataset. For this purpose, you will be using a data subsample from the Kaggle "Two sigma connect: rental listing inquiries" competition.

- Before proceeding with any imputing you need to know the number of missing values for each of the features. Moreover, if the feature has missing values, you should explore the type of this feature.

### Instructions 1/2
-    Read the "twosigma_train.csv" file using pandas.
-    Find the number of missing values in each column.

In [ ]:
# Read dataframe
twosigma = pd.read_csv("twosigma_train.csv")

# Find the number of missing values in each column
print(twosigma.isnull().sum())

### Instructions 2/2
-    Select the columns with the missing values and look at the head of the DataFrame.

In [ ]:
# Read DataFrame
twosigma = pd.read_csv('twosigma_train.csv')

# Find the number of missing values in each column
print(twosigma.isnull().sum())

# Look at the columns with the missing values
print(twosigma[["building_id", "price"]].head())

## Impute missing data

- You've found that "price" and "building_id" columns have missing values in the Rental Listing Inquiries dataset. So, before passing the data to the models you need to impute these values.

- Numerical feature "price" will be encoded with a mean value of non-missing prices.

- Imputing categorical feature "building_id" with the most frequent category is a bad idea, because it would mean that all the apartments with a missing "building_id" are located in the most popular building. The better idea is to impute it with a new category.

- The DataFrame rental_listings with competition data is read for you.

### Instructions 1/2
-        Create a SimpleImputer object with "mean" strategy.
-        Impute missing prices with the mean value.

In [ ]:
# Import SimpleImputer
from sklearn.impute import SimpleImputer

# Create mean imputer
mean_imputer = SimpleImputer(strategy='mean')

# Price imputation
rental_listings[['price']] = mean_imputer.fit_transform(rental_listings[["price"]])

### Instructions 2/2
-        Create an imputer with "constant" strategy. Use "MISSING" as fill_value.
-        Impute missing buildings with a constant value.

In [ ]:
# Import SimpleImputer
from sklearn.impute import SimpleImputer

# Create constant imputer
constant_imputer = SimpleImputer(strategy="constant", fill_value='MISSING')

# building_id imputation
rental_listings[['building_id']] = constant_imputer.fit_transform(rental_listings[["building_id"]])